
### АвиаПарсинг

**Описание**:

Данные о перелетах и их ценах по России за 15 мая - 30. Парсинг данных осуществляется с сайта 'Авиасеилс' с помощью requests и fetch(Node.js). Анализируется средний процент роста цен на авиабилеты с приближением летнего сезона, а также выделение топ 5 самых дешевых перелетов из Московских аэропортов.

In [17]:
import requests
import pandas as pd
import plotly.express as px
import folium
from folium.plugins import PolyLineTextPath
from datetime import datetime

# Загрузка данных

In [2]:
df = pd.DataFrame(columns=['City'])
def fetch(url, params):
    headers = params['headers']
    body = params['body']
    method = params['method']
    if method == 'POST':
        return requests.post(url, headers=headers, data=body)
    if method == 'GET':
        return requests.post(url, headers=headers)
def update_dataframe(df, current_date, directions):
    price_column = f'Price({current_date})'

    if price_column not in df.columns:
        df[price_column] = None

    for offer in directions:
        try:
            city = offer['destination']['translations']['ru']['su']
            price = offer['price']['value']

            if price is None:
                break

        except (KeyError, TypeError):
            break

        if city in df['City'].values:
            df.loc[df['City'] == city, price_column] = price
        else:
            new_row = {'City': city, price_column: price}
            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

    return df

In [3]:
May15 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "82d6745acc345344127f7f06ecb8cf05",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-05-15\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

May16 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "861da8fc77de10256cd52069ed088dda",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-05-16\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

May17 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-05-17\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

May18 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-05-18\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

May19 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-05-19\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

May20 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-05-20\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

May21 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-05-21\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

May22 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-05-22\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

May23 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-05-23\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

May24 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-05-24\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

May25 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-05-25\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

May26 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-05-26\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

May27 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-05-27\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

May28 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-05-28\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

May29 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-05-29\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

May30 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-05-30\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

May31 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-05-31\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})


June01 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-01\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})
June02 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-02\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June03 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-03\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June04 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-04\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June05 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-05\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June06 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-06\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June07 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-07\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June08 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-08\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June09 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "aa2a5645c9a71bd992c98657dd9c3db8",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-09\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June10 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-10\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June11 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-11\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June12 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-12\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June13 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-13\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June14 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-14\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June15 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-15\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June16 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-16\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June17 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-17\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June18 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-18\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June19 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-19\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June20 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-20\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June21 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-21\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June22 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-22\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June23 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-23\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June24 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-24\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June25 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-25\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June26 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-26\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June27 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-27\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June28 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-28\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June29 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-29\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

June30 = fetch("https://ariadne.aviasales.com/api/gql", {
  "headers": {
    "accept": "application/json",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "x-auid": "ZlJVVWcZOIhFNWJyWDJPAg==",
    "x-request-id": "ea320c56d40cc6f1b82978f6f45ce472",
    "x-user-platform": "web",
    "Referer": "https://www.aviasales.ru/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  },
  "body": "{\"query\":\"\\n  query GetCountryBestDirectionsV2(\\n    $input: CountryBestDirectionsV2Input!\\n    $brand: Brand!\\n    $language: String!\\n  ) {\\n    country_best_directions_v2(\\n      input: $input\\n      brand: $brand\\n    ) {\\n      country {\\n        iata\\n        translations(filters: { locales: [$language] })\\n      }\\n      total\\n      directions {\\n        city_info {\\n          image_url\\n        }\\n        destination {\\n          iata\\n          translations(filters: { locales: [$language] })\\n        }\\n        price {\\n          value\\n          currency\\n        }\\n      } \\n    }\\n  }\\n\",\"variables\":{\"brand\":\"AS\",\"language\":\"ru\",\"input\":{\"origin_iata\":\"MOW\",\"origin_type\":\"CITY\",\"passport_country\":\"RU\",\"trip_class\":\"Y\",\"one_way\":true,\"currency\":\"rub\",\"market\":\"ru\",\"dates\":{\"depart_dates\":[\"2025-06-30\"]},\"paging\":{\"offset\":0,\"limit\":100},\"filters\":{\"direct\":false,\"no_visa_at_transfer\":false,\"with_baggage\":false},\"destination_country_iata\":\"RU\"}},\"operation_name\":\"country_best_directions_v2\"}",
  "method": "POST"
})

In [4]:
#Май 15(Дата сбора 14.05.2025)
#Можно было бы написать цикл, но я уже реализовал запарным, но читабельным способом
May15.status_code
directions = May15.json()['data']['country_best_directions_v2']['directions']
current_date = 'May15'
df = update_dataframe(df, current_date, directions)
#Май 16
May16.status_code
directions = May16.json()['data']['country_best_directions_v2']['directions']
current_date = 'May16'
df = update_dataframe(df, current_date, directions)
#Май 17
May17.status_code
directions = May17.json()['data']['country_best_directions_v2']['directions']
current_date = 'May17'
df = update_dataframe(df, current_date, directions)
#Май 18
May18.status_code
directions = May18.json()['data']['country_best_directions_v2']['directions']
current_date = 'May18'
df = update_dataframe(df, current_date, directions)
#Май 19
May19.status_code
directions = May19.json()['data']['country_best_directions_v2']['directions']
current_date = 'May19'
df = update_dataframe(df, current_date, directions)
#Май 20
May20.status_code
directions = May20.json()['data']['country_best_directions_v2']['directions']
current_date = 'May20'
df = update_dataframe(df, current_date, directions)
#Май 21
May21.status_code
directions = May21.json()['data']['country_best_directions_v2']['directions']
current_date = 'May21'
df = update_dataframe(df, current_date, directions)
#Май 22
May22.status_code
directions = May22.json()['data']['country_best_directions_v2']['directions']
current_date = 'May22'
df = update_dataframe(df, current_date, directions)
#Май 23
May23.status_code
directions = May23.json()['data']['country_best_directions_v2']['directions']
current_date = 'May23'
df = update_dataframe(df, current_date, directions)
#Май 24
May24.status_code
directions = May24.json()['data']['country_best_directions_v2']['directions']
current_date = 'May24'
df = update_dataframe(df, current_date, directions)
#Май 25
May25.status_code
directions = May25.json()['data']['country_best_directions_v2']['directions']
current_date = 'May25'
df = update_dataframe(df, current_date, directions)
#Май 26
May26.status_code
directions = May26.json()['data']['country_best_directions_v2']['directions']
current_date = 'May26'
df = update_dataframe(df, current_date, directions)
#Май 27
May27.status_code
directions = May27.json()['data']['country_best_directions_v2']['directions']
current_date = 'May27'
df = update_dataframe(df, current_date, directions)
#Май 28
May28.status_code
directions = May28.json()['data']['country_best_directions_v2']['directions']
current_date = 'May28'
df = update_dataframe(df, current_date, directions)
#Май 29
May29.status_code
directions = May29.json()['data']['country_best_directions_v2']['directions']
current_date = 'May29'
df = update_dataframe(df, current_date, directions)
#Май 30
May30.status_code
directions = May30.json()['data']['country_best_directions_v2']['directions']
current_date = 'May30'
df = update_dataframe(df, current_date, directions)
#Май 31
May31.status_code
directions = May31.json()['data']['country_best_directions_v2']['directions']
current_date = 'May31'
df = update_dataframe(df, current_date, directions)

#Июнь 01
June01.status_code
directions = June01.json()['data']['country_best_directions_v2']['directions']
current_date = 'June01'
df = update_dataframe(df, current_date, directions)
#Июнь 02
June02.status_code
directions = June02.json()['data']['country_best_directions_v2']['directions']
current_date = 'June02'
df = update_dataframe(df, current_date, directions)
#Июнь 03
June03.status_code
directions = June03.json()['data']['country_best_directions_v2']['directions']
current_date = 'June03'
df = update_dataframe(df, current_date, directions)
#Июнь 04
June04.status_code
directions = June04.json()['data']['country_best_directions_v2']['directions']
current_date = 'June04'
df = update_dataframe(df, current_date, directions)
#Июнь 05
June05.status_code
directions = June05.json()['data']['country_best_directions_v2']['directions']
current_date = 'June05'
df = update_dataframe(df, current_date, directions)
#Июнь 06
June06.status_code
directions = June06.json()['data']['country_best_directions_v2']['directions']
current_date = 'June06'
df = update_dataframe(df, current_date, directions)
#Июнь 07
June07.status_code
directions = June07.json()['data']['country_best_directions_v2']['directions']
current_date = 'June07'
df = update_dataframe(df, current_date, directions)
#Июнь 08
June08.status_code
directions = June08.json()['data']['country_best_directions_v2']['directions']
current_date = 'June08'
df = update_dataframe(df, current_date, directions)
#Июнь 09
June09.status_code
directions = June09.json()['data']['country_best_directions_v2']['directions']
current_date = 'June09'
df = update_dataframe(df, current_date, directions)
#Июнь 10
June10.status_code
directions = June10.json()['data']['country_best_directions_v2']['directions']
current_date = 'June10'
df = update_dataframe(df, current_date, directions)
#Июнь 11
June11.status_code
directions = June11.json()['data']['country_best_directions_v2']['directions']
current_date = 'June11'
df = update_dataframe(df, current_date, directions)
#Июнь 12
June12.status_code
directions = June12.json()['data']['country_best_directions_v2']['directions']
current_date = 'June12'
df = update_dataframe(df, current_date, directions)
#Июнь 13
June13.status_code
directions = June13.json()['data']['country_best_directions_v2']['directions']
current_date = 'June13'
df = update_dataframe(df, current_date, directions)
#Июнь 14
June14.status_code
directions = June14.json()['data']['country_best_directions_v2']['directions']
current_date = 'June14'
df = update_dataframe(df, current_date, directions)
#Июнь 15
June15.status_code
directions = June15.json()['data']['country_best_directions_v2']['directions']
current_date = 'June15'
df = update_dataframe(df, current_date, directions)
#Июнь 16
June16.status_code
directions = June16.json()['data']['country_best_directions_v2']['directions']
current_date = 'June16'
df = update_dataframe(df, current_date, directions)
#Июнь 17
June17.status_code
directions = June17.json()['data']['country_best_directions_v2']['directions']
current_date = 'June17'
df = update_dataframe(df, current_date, directions)
#Июнь 18
June18.status_code
directions = June18.json()['data']['country_best_directions_v2']['directions']
current_date = 'June18'
df = update_dataframe(df, current_date, directions)
#Июнь 19
June19.status_code
directions = June19.json()['data']['country_best_directions_v2']['directions']
current_date = 'June19'
df = update_dataframe(df, current_date, directions)
#Июнь 20
June20.status_code
directions = June20.json()['data']['country_best_directions_v2']['directions']
current_date = 'June20'
df = update_dataframe(df, current_date, directions)
#Июнь 21
June21.status_code
directions = June21.json()['data']['country_best_directions_v2']['directions']
current_date = 'June21'
df = update_dataframe(df, current_date, directions)
#Июнь 22
June22.status_code
directions = June22.json()['data']['country_best_directions_v2']['directions']
current_date = 'June22'
df = update_dataframe(df, current_date, directions)
#Июнь 23
June23.status_code
directions = June23.json()['data']['country_best_directions_v2']['directions']
current_date = 'June23'
df = update_dataframe(df, current_date, directions)
#Июнь 24
June24.status_code
directions = June24.json()['data']['country_best_directions_v2']['directions']
current_date = 'June24'
df = update_dataframe(df, current_date, directions)
#Июнь 25
June25.status_code
directions = June25.json()['data']['country_best_directions_v2']['directions']
current_date = 'June25'
df = update_dataframe(df, current_date, directions)
#Июнь 26
June26.status_code
directions = June26.json()['data']['country_best_directions_v2']['directions']
current_date = 'June26'
df = update_dataframe(df, current_date, directions)
#Июнь 27
June27.status_code
directions = June27.json()['data']['country_best_directions_v2']['directions']
current_date = 'June27'
df = update_dataframe(df, current_date, directions)
#Июнь 28
June28.status_code
directions = June28.json()['data']['country_best_directions_v2']['directions']
current_date = 'June28'
df = update_dataframe(df, current_date, directions)
#Июнь 29
June29.status_code
directions = June29.json()['data']['country_best_directions_v2']['directions']
current_date = 'June29'
df = update_dataframe(df, current_date, directions)
#Июнь 30
June30.status_code
directions = June30.json()['data']['country_best_directions_v2']['directions']
current_date = 'June30'
df = update_dataframe(df, current_date, directions)

## Очистка данных и подготовка к анализу

In [5]:
df.head()

,City,Price(May15),Price(May16),Price(May17),Price(May18),Price(May19),Price(May20),Price(May21),Price(May22),Price(May23),...,Price(June21),Price(June22),Price(June23),Price(June24),Price(June25),Price(June26),Price(June27),Price(June28),Price(June29),Price(June30)
0,Мурманск,2354,2354,2404,2393.48,2411,2389,2400,2954,5498,...,7851.36,7150,7256,6770.72,5952,6779,8523,7426,10811.12,8505
1,Уфа,5394,3130,2658,2950,3275,3976,3130,4311,5352,...,6191.59,6397,6180,6291,6397,6397,6397,7435,7398,6384.95
2,Архангельск,5491,5491,4250,5159.66,5159.66,5911,4250,5159.66,7980,...,10068.41,6563.4,5940.41,5158.17,5940.41,5941.87,6565.29,9644,6565.29,5931
3,Сыктывкар,6267,6851,6766,6423.03,5501,5542,5572,6336.59,7808,...,9411.62,9377,8681.88,8012.69,7963.61,6795.72,9818.38,9465.72,12516.43,9382.86
4,Самара,6665,5619,3960,3859,4376,3824,3846,3804,4148,...,5524.34,5516.09,5504,5501,5529,6665,6665,6597.35,6569,6585


Уберем все Nan на среднюю цену билета

In [6]:
cnt = [col for col in df.columns if 'Price' in col]
for col in cnt:
    df[col] = df[col].fillna(df[col].mean())

<ipython-input-6-eb20e13be2dd>:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(df[col].mean())


## Анализ и визуализация

In [8]:
'''
price_cols = [col for col in df.columns if col.startswith('Price')]
df_prices = df[price_cols]

pct = df_prices.pct_change(axis=1) * 100
pct = pct.iloc[:, 1:]
df['Средний процент изменения (%)'] = pct.mean(axis=1).round(2)

result = df[['City', 'Средний процент изменения (%)']]

overall_average = df['Средний процент изменения (%)'].mean().round(2)

print("Таблица средних процентов изменения цен по городам:")
print(result.to_string(index=False))
print(f"\nОбщий средний процент изменения цен: {overall_average}%")
'''
# Вывод: среднее повышение цены ~5%
print('# Вывод: среднее повышение цены к летнему сезону ~5%')

# Вывод: среднее повышение цены к летнему сезону ~5%


Визуализация изменения цен на авиабилеты за май-июнь

In [9]:
df_melted = df.melt(
    id_vars='City',
    var_name='Date',
    value_name='Price'
)

df_melted[['Month', 'Day']] = df_melted['Date'].str.extract(r'(May|June)(\d+)')
df_melted['Date'] = df_melted.apply(
    lambda x: f"2025-{x['Month']}-{x['Day']}",
    axis=1
)
df_melted['Date'] = pd.to_datetime(df_melted['Date'], format='%Y-%B-%d')
df_melted = df_melted.drop(['Month', 'Day'], axis=1)
df_melted = df_melted.sort_values('Date')
fig = px.line(
    df_melted,
    x='Date',
    y='Price',
    color='City',
    title='Динамика изменения цен на авиабилеты по городам с вылетом из Москвы (Май-Июнь 2025)',
    labels={'Price': 'Цена', 'Date': 'Дата'},
    template='plotly_white'
)
fig.update_xaxes(
    dtick="D1",
    tickformat="%d-%b",
    rangeslider_visible=True
)

fig.show()

Узнаем какие направления наиболее дорожают к летнему сезону

In [10]:
df_melted = df.melt(id_vars='City', var_name='Date', value_name='Price')
df_melted[['Month', 'Day']] = df_melted['Date'].str.extract(r'(May|June)(\d+)')
df_melted['Day'] = df_melted['Day'].str.zfill(2)
df_melted['Date'] = pd.to_datetime(
    df_melted.apply(lambda x: f"2023-{x['Month']}-{x['Day']}", axis=1),
    format='%Y-%B-%d'
)
df_melted = df_melted.drop(['Month', 'Day'], axis=1).sort_values('Date')

price_growth = df_melted.groupby('City').apply(
    lambda x: (x['Price'].iloc[-1] - x['Price'].iloc[0]) / x['Price'].iloc[0] * 100
).reset_index(name='Growth%')
top_cities = price_growth.nlargest(5, 'Growth%')['City'].tolist()
filtered_data = df_melted[df_melted['City'].isin(top_cities)]
fig = px.line(
    filtered_data,
    x='Date',
    y='Price',
    color='City',
    title='Топ-5 перелетов из Москвы с самым быстрым ростом цен (Май-Июнь 2025)',
    labels={'Price': 'Цена', 'Date': 'Дата'},
    template='plotly_white'
)

fig.update_xaxes(
    dtick="D3",
    tickformat="%d-%b",
    rangeslider_visible=True
)

fig.show()

<ipython-input-10-fb281289b299>:10: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



Визуализируем данную зависимость на карте

In [12]:
import folium
from folium.plugins import PolyLineTextPath
from datetime import datetime
df_melted = df.melt(id_vars='City', var_name='Date', value_name='Price')
df_melted[['Month', 'Day']] = df_melted['Date'].str.extract(r'(May|June)(\d+)')
df_melted['Day'] = df_melted['Day'].str.zfill(2)
df_melted['Date'] = pd.to_datetime(
    df_melted.apply(lambda x: f"2023-{x['Month']}-{x['Day']}", axis=1),
    format='%Y-%B-%d'
)
df_melted = df_melted.drop(['Month', 'Day'], axis=1).sort_values('Date')

price_growth = df_melted.groupby('City').apply(
    lambda x: (x['Price'].iloc[-1] - x['Price'].iloc[0]) / x['Price'].iloc[0] * 100
).reset_index(name='Growth%')

top_cities = price_growth.nlargest(5, 'Growth%')['City'].tolist()
latest_prices = df_melted.groupby('City')['Price'].last().to_dict()
#Возможные города лидеры по росту цены
city_coords = {
    'Москва': [55.7558, 37.6173],
    'Калининград': [54.7104, 20.4522],
    'Певек': [69.7008, 170.3133],
    'Челябинск': [55.1644, 61.4368],
    'Мурманск': [68.9585, 33.0823],
    'Анадырь': [64.73, 177.51],
    'Калуга': [64.73, 177.51],
    'Ханты-Мансийск': [61.0042, 69.0019],
}

m = folium.Map(location=[55, 100], zoom_start=3, tiles='CartoDB positron')
folium.Marker(
    location=city_coords['Москва'],
    popup=f'Москва<br>Цена: {latest_prices.get("Москва", "N/A")} руб.',
    icon=folium.Icon(
        color='red',
        icon='plane',
        prefix='fa',
        icon_size=(40, 40)
    )
).add_to(m)

for city in top_cities:
    if city != 'Москва' and city in city_coords:
        folium.Marker(
            location=city_coords[city],
            popup=f'{city}<br>Цена: {latest_prices.get(city, "N/A")} руб.',
            icon=folium.Icon(
                color='blue',
                icon='star',
                prefix='fa'
            )
        ).add_to(m)

        folium.map.Marker(
            [city_coords[city][0] - 1.5, city_coords[city][1]],
            icon=folium.DivIcon(
                icon_size=(150, 30),
                icon_anchor=(0,0),
                html=f'<div style="font-size: 14pt; font-weight: bold; color: darkgreen;">{latest_prices.get(city, "N/A")} руб.</div>'
            )
        ).add_to(m)

colors = ['#FF0000', '#00FF00', '#0000FF', '#FFA500', '#800080']

def add_arrow_line(map_obj, start, end, color, legend_name):
    line = folium.PolyLine(
        locations=[start, end],
        color=color,
        weight=2
    )
    line.add_to(map_obj)
    PolyLineTextPath(
        line,
        '►',
        repeat=False,
        offset=7,
        attributes={'fill': color, 'font-size': '20'}
    ).add_to(map_obj)

for idx, city in enumerate(top_cities):
    if city != 'Москва' and city in city_coords:
        add_arrow_line(
            m,
            city_coords['Москва'],
            city_coords[city],
            colors[idx % len(colors)],
            f'Москва → {city}'
        )

legend_html = '''
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 250px;
    height: auto;
    border:2px solid grey;
    z-index:9999;
    font-size:14px;
    background-color:white;
    padding: 10px;
    ">
    <b>Топ-5 перелетов из Москвы с самым быстрым ростом цен (Май-Июнь 2025):</b><br>
    {}
</div>'''.format('<br>'.join([f'• {city}: {latest_prices.get(city, "N/A")} руб.' for city in top_cities]))

m.get_root().html.add_child(folium.Element(legend_html))
m

<ipython-input-12-fc9e2102a661>:13: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



Также отображу самые дешевые перелеты из Москвы

In [13]:
df_melted = df.melt(id_vars='City', var_name='Date', value_name='Price')
df_melted[['Month', 'Day']] = df_melted['Date'].str.extract(r'(May|June)(\d+)')
df_melted['Day'] = df_melted['Day'].str.zfill(2)
df_melted['Date'] = pd.to_datetime(
    df_melted.apply(lambda x: f"2023-{x['Month']}-{x['Day']}", axis=1),
    format='%Y-%B-%d'
)
df_melted = df_melted.drop(['Month', 'Day'], axis=1).sort_values('Date')

price_stats = df_melted.groupby('City')['Price'].agg(['min', 'mean']).reset_index()
top_cities = price_stats.nsmallest(5, 'min')['City'].tolist()

city_coords = {
    'Москва': [55.7558, 37.6173],
    'Калининград': [54.7104, 20.4522],
    'Певек': [69.7008, 170.3133],
    'Челябинск': [55.1644, 61.4368],
    'Мурманск': [68.9585, 33.0823],
    'Ярославль': [57.63, 39.87],
    'Псков': [57.81, 28.35],
    'Санкт-Петербург': [59.9386, 30.3141],
    'Чебоксары': [56.13, 47.25],
    'Тамбов': [52.73, 41.44],
    'Пермь': [58.01, 56.25],

}

m = folium.Map(location=[55, 100], zoom_start=3, tiles='CartoDB positron')

cheap_style = {
    'icon': 'eur',
    'color': 'green',
    'prefix': 'fa'
}

for city in top_cities:
    if city in city_coords:
        min_price = price_stats[price_stats['City'] == city]['min'].values[0]

        folium.Marker(
            location=city_coords[city],
            popup=f'''
                <b>{city}</b><br>
                Мин. цена: {min_price} руб.<br>
                Средняя: {price_stats[price_stats['City'] == city]['mean'].values[0]:.0f} руб.
            ''',
            icon=folium.Icon(**cheap_style)
        ).add_to(m)

if 'Москва' in city_coords:
    for city in top_cities:
        if city != 'Москва' and city in city_coords:
            folium.PolyLine(
                locations=[city_coords['Москва'], city_coords[city]],
                color='lightgreen',
                weight=1.5,
                opacity=0.7
            ).add_to(m)

legend_html = f'''
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 300px;
    background: white;
    padding: 10px;
    border: 2px solid grey;
    z-index: 9999;">
    <h4>Топ-5 дешевых направлений</h4>
    {"<br>".join([
        f'• {city}: {price_stats[price_stats["City"] == city]["min"].values[0]} руб.'
        for city in top_cities
    ])}
</div>'''

m.get_root().html.add_child(folium.Element(legend_html))

filtered_data = df_melted[df_melted['City'].isin(top_cities)]
fig = px.line(
    filtered_data,
    x='Date',
    y='Price',
    color='City',
    title='Топ-5 самых дешевых перелетов из Москвы (Май-Июнь 2025)',
    labels={'Price': 'Цена, руб.'},
    template='plotly_white'
)

fig.update_layout(hovermode='x unified')
fig.show()

m

## Выводы

Рассмотрев полученные данные, я могу сказать что рост цен к летнему периоду составлет примерно за месяц 5-6%, когда инфляция за месяц примерно 1-2%, отсюда мы можем сделать вывод что в связи с повышением количества отпусков/командировок/каникул, цены на авиабилеты возрастают, и лидерами в приросте цен являются Мурманск, Певецк и Калининград, два города северных и один на дальнем востоке, что окак раз оправдывает многие нужны граждон в более холодном климате или же в курортном районе с морем. Также был проведен анализ самых дешевых перелетов, а именно в Ярославль, Санкт Петербург и Пермь, данные города являются ближайшими к Москве что подтверждает их относительно низкую стоймость.

## Обсуждение

В рамках иследования, я сделал парсинг данных за короткий промежуток времени ~месяц, где явно виден прирост стоимости за авиабмлеты. Были выделен данные о самых дешевых перелетах, а также перелетах на цена которых возрастает к летнему сезону, был выявлен процент этого самого роста, что позволяет узнать примерную цену билета через несколько месяцев.
-Не удалось сделать более маштабное иследование( за год-два ), что показало бы реальный процент роста цен.
-Опять же увеличить количество данных, а также узнать новые параменты учитывавюшие компанию, время перелета, частота зареджек по определенному направлению или по определенной компании.
Покупателям авиабилетов для понимания лучшей компании(количестов задержек рейса и цена на аивабилеты по сравнению с другими компаниями), примерного времени перелета. Планирующим узнать примерные цены на перелеты с учетом повышения цен.